# **Flying Object Detection and Tracking for Defending Designated Zones with YOLO and DeepSORT**

This demo project focuses on developing an AI-powered system designed to detect, track, and classify flying objects in real time, with the primary goal of defending a designated Region of Interest (ROI) from a specific type of object—in this example, birds. By combining deep learning models and tracking algorithms, the system processes video feeds to differentiate between target objects (birds) and non-target objects (such as aircraft).

The system's core objective is to monitor and respond to specific objects entering the ROI while ignoring others, ensuring a targeted and efficient defense mechanism that can be adapted to various contexts. It leverages YOLO (You Only Look Once) for object detection and DeepSORT (Simple Online and Realtime Tracking with a Deep Association Metric) for tracking across frames.

🚀 𝐊𝐞𝐲 𝐅𝐞𝐚𝐭𝐮𝐫𝐞𝐬:

𝐓𝐚𝐫𝐠𝐞𝐭𝐞𝐝 𝐃𝐞𝐭𝐞𝐜𝐭𝐢𝐨𝐧, 𝐃𝐢𝐬𝐜𝐫𝐢𝐦𝐢𝐧𝐚𝐭𝐢𝐨𝐧, 𝐚𝐧𝐝 𝐓𝐫𝐚𝐜𝐤𝐢𝐧𝐠  
Detects and tracks specific objects in real time, distinguishing them from other non-relevant objects.  
𝐍𝐨𝐧-𝐈𝐧𝐭𝐞𝐫𝐚𝐜𝐭𝐢𝐨𝐧 𝐰𝐢𝐭𝐡 𝐍𝐨𝐧-𝐓𝐚𝐫𝐠𝐞𝐭𝐬  
Identifies non-target objects without tracking or interacting with them, allowing resources to focus on the designated threat.  
𝐑𝐎𝐈-𝐁𝐚𝐬𝐞𝐝 𝐑𝐞𝐬𝐩𝐨𝐧𝐬𝐞  
Triggers specific responses when the target object enters the ROI. For a playful experience, the system “feeds” birds with virtual peanuts in a hostile manner, creating a fun, game-like interaction.

📌 𝐏𝐨𝐭𝐞𝐧𝐭𝐢𝐚𝐥 𝐀𝐩𝐩𝐥𝐢𝐜𝐚𝐭𝐢𝐨𝐧𝐬:

𝐃𝐫𝐨𝐧𝐞 𝐃𝐞𝐟𝐞𝐧𝐬𝐞 𝐒𝐲𝐬𝐭𝐞𝐦𝐬: Prevent unauthorized drones from entering restricted areas, such as military zones or private property.  
𝐈𝐧𝐝𝐮𝐬𝐭𝐫𝐢𝐚𝐥 𝐒𝐢𝐭𝐞 𝐒𝐞𝐜𝐮𝐫𝐢𝐭𝐲: Detect and track specific intrusions near sensitive equipment or infrastructure.  
𝐄𝐯𝐞𝐧𝐭 𝐒𝐮𝐫𝐯𝐞𝐢𝐥𝐥𝐚𝐧𝐜𝐞: Monitor large gatherings for unauthorized flying objects like drones.  
𝐁𝐨𝐫𝐝𝐞𝐫 𝐏𝐫𝐨𝐭𝐞𝐜𝐭𝐢𝐨𝐧: Track specific aircraft crossing borders in restricted zones.  
𝐖𝐢𝐥𝐝𝐥𝐢𝐟𝐞 𝐏𝐫𝐨𝐭𝐞𝐜𝐭𝐢𝐨𝐧: Safeguard sensitive areas from specific animals, e.g., protecting crops from birds or other wildlife.  
𝐀𝐢𝐫 𝐓𝐫𝐚𝐟𝐟𝐢𝐜 𝐒𝐚𝐟𝐞𝐭𝐲: Defend airspace from unauthorized drones or birds near airports.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install deep_sort_realtime opencv-python-headless
!pip install opencv-python
!pip install ultralytics

In [ ]:
import cv2
import numpy as np
from deep_sort_realtime.deepsort_tracker import DeepSort
from ultralytics import YOLO
import random

# Load YOLOv8 model
model = YOLO('yolov8n.pt')

# Initialize DeepSORT tracker
tracker = DeepSort(max_age=5)

# Load video
video_path = '/content/drive/MyDrive/path_to_seagulls_video/seagulls_video.mp4' # Replace with your path
cap = cv2.VideoCapture(video_path)

# Get video properties for output
fps = cap.get(cv2.CAP_PROP_FPS)
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

# Set the Region of Interest (ROI)
roi_width = 950
roi_height = 2 * frame_height // 5
roi_top_left = (100, 100)
roi_bottom_right = (roi_width, roi_height)

line_thickness = 2
output_path = '/content/drive/MyDrive/Peanut Feeder for Birds.mp4' # Replace with your path
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_path, fourcc, fps, (frame_width, frame_height))

# Load plane image with alpha channel (transparency)
plane_img = cv2.imread('/content/drive/MyDrive/path_to_airplane_picture/airplane_picture.png', cv2.IMREAD_UNCHANGED) # Replace with your path (the picture needs to be in .png format).
plane_width = 100
plane_height = 50
plane_x = 1200  # Starting x position
plane_y = 200  # Starting y position

# Load helicopter image with alpha channel (transparency)
helicopter_img = cv2.imread('/content/drive/MyDrive/path_to_helicopter_picture/helicopter_picture.png', cv2.IMREAD_UNCHANGED) # Replace with your path (the picture needs to be in .png format).
helicopter_width = 140
helicopter_height = 92
helicopter_x = 160  # Starting x position
helicopter_y = 450  # Starting y position

# Initialize peanut shot state
peanut_shot_flash_counter = 0  # Counter for flashing "Peanut Shot!" text
peanut_flash_duration = 15  # Number of frames for which the text is visible

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Run YOLOv8 object detection
    results = model(frame)
    confidence_threshold = 0.4
    detections = []
    for result in results:
        for box in result.boxes:
            class_id = int(box.cls[0])
            confidence = box.conf[0]
            if confidence > confidence_threshold:
                x1, y1, x2, y2 = map(int, box.xyxy[0])
                if class_id == 14:  # class_id 14 is for birds
                    width = x2 - x1
                    height = y2 - y1
                    detections.append([[x1, y1, width, height], confidence, class_id])

    if detections:
        tracks = tracker.update_tracks(detections, frame=frame)
        for track in tracks:
            bbox = track.to_tlbr()
            track_id = track.track_id
            x1, y1, x2, y2 = map(int, bbox)
            label = "Bird"
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), line_thickness)
            cv2.putText(frame, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

            if (x1 < roi_bottom_right[0] and x2 > roi_top_left[0] and
                y1 < roi_bottom_right[1] and y2 > roi_top_left[1]):
                peanut_start = (frame_width // 2, frame_height - 50)
                peanut_end = ((x1 + x2) // 2, (y1 + y2) // 2)
                cv2.arrowedLine(frame, peanut_start, peanut_end, (0, 165, 255), 2, tipLength=0.15)

                # Flashing effect for "Peanut Shot!" text
                if peanut_shot_flash_counter == 0:
                    cv2.putText(frame, "Peanut Shot!", (425, 70), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
                    peanut_shot_flash_counter = peanut_flash_duration
                elif peanut_shot_flash_counter > 0:
                    peanut_shot_flash_counter -= 1

    plane_x -= 2  # Move plane slowly to the left

    # Resize plane image to fit within the frame
    plane_img_resized = cv2.resize(plane_img, (plane_width, plane_height))

    # Ensure it matches the color channels (BGR from RGBA)
    if plane_img_resized.shape[2] == 4:  # If it has an alpha channel
        # Split the image into BGRA channels (BGR + Alpha)
        bgr_plane = plane_img_resized[:, :, :3]
        alpha_plane = plane_img_resized[:, :, 3]

        # Resize and position the plane image in the frame
        x_end = plane_x + plane_width
        y_end = plane_y + plane_height

        # Create a mask based on the alpha channel
        mask = alpha_plane / 255.0
        mask_inv = 1.0 - mask

        # Blend the frame and plane using the mask
        for c in range(0, 3):  # For each channel (BGR)
            frame[plane_y:y_end, plane_x:x_end, c] = (frame[plane_y:y_end, plane_x:x_end, c] * mask_inv +
                                                      bgr_plane[:, :, c] * mask)

    # Slowly move the helicopter upwards and to the right
    helicopter_y -= random.choices([0, 1, -1], weights=[3, 3, 1], k=1)[0]
    helicopter_x -= random.choices([0, 1, -1], weights=[5, 1, 3], k=1)[0]

    # Resize helicopter image to fit within the frame
    helicopter_img_resized = cv2.resize(helicopter_img, (helicopter_width, helicopter_height))

    # Ensure it matches the color channels (BGR from RGBA)
    if helicopter_img_resized.shape[2] == 4:  # If it has an alpha channel
        # Split the image into BGRA channels (BGR + Alpha)
        bgr_helicopter = helicopter_img_resized[:, :, :3]
        alpha_helicopter = helicopter_img_resized[:, :, 3]

        # Resize and position the helicopter image in the frame
        x_end = helicopter_x + helicopter_width
        y_end = helicopter_y + helicopter_height

        # Create a mask based on the alpha channel
        mask = alpha_helicopter / 255.0
        mask_inv = 1.0 - mask

        # Blend the frame and helicopter using the mask
        for c in range(0, 3):  # For each channel (BGR)
            frame[helicopter_y:y_end, helicopter_x:x_end, c] = (frame[helicopter_y:y_end, helicopter_x:x_end, c] * mask_inv +
                                                                  bgr_helicopter[:, :, c] * mask)

    cv2.rectangle(frame, roi_top_left, roi_bottom_right, (255, 0, 0), line_thickness)
    out.write(frame)

cap.release()
out.release()
print(f"Processed video saved to: {output_path}")
